In [ ]:
#OLLAMAPool Server Prototype Code
import os
EndPoint_Queries=os.environ.get('EndPoint_Queries')
Endpoint_Results=os.environ.get('Endpoint_Results')
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_Queries is None:
    raise ValueError("EndPoint_Queries is not set")
if Endpoint_Results is None:
    raise ValueError("Endpoint_Results is not set")
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
print("Environment Variables are set OK")


In [ ]:
#Example Use case -> character description generator for fantasy RPG
#Takes a random dictionary of character traits and generates a written character description

races=["elf","human","dwarf","halfling","gnome"]
complections=["pale","dark","fair","ruddy","swarthy"]
heights=["tall","short","average"]
builds=["slender","stocky","muscular","portly"]
features=["sharp","soft","angular","round"]
eyes=["bright","dark","piercing","dull"]
hairs=["long","short","curly","straight","wavy"]
clothes=["fine","simple","elegant","gaudy","tattered"]
sophistication=["refined","crude","simple","complex"]
mannerisms=["graceful","clumsy","boisterous","quiet"]
virtues=["honest","loyal","brave","generous","kind"]


import random

#Come up with a random dictionary of character traits
def GenerateCharacterDescDictionary():
    traits={}
    race=random.choice(races)
    complection=random.choice(complections)
    height=random.choice(heights)
    build=random.choice(builds)
    feature=random.choice(features)
    eye=random.choice(eyes)
    hair=random.choice(hairs)
    clothe=random.choice(clothes)
    sophisticate=random.choice(sophistication)
    mannerism=random.choice(mannerisms)
    virtue=random.choice(virtues)
    traits={"race":race,"complection":complection,"height":height,"build":build,"feature":feature,"eye":eye,"hair":hair,"clothes":clothe,"sophistication":sophisticate,"mannerism":mannerism,"virtue":virtue}
    return traits

#Turn the dictionary into a line separated string
def GenerateCharacterDescList(traits):
    desc="They have the followng traits:\n"
    for key in traits:
        desc+=key+": "+traits[key]+"\n"
    return desc
    
            
    
#Test code
traits=GenerateCharacterDescDictionary()
print(GenerateCharacterDescList(traits))


In [ ]:
#Produce sample character descriptions
totalRecords=10
requestTexts=[]
for i in range(totalRecords):
    traits=GenerateCharacterDescDictionary()
    requestTexts.append(GenerateCharacterDescList(traits))
requestTexts

In [4]:
#Busines Classes
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage

class LLMRequest:
    def __init__(self,Model:str="",systemMessage:str="",query:str=""):
        self.UUID=str(uuid.uuid4())
        self.Model=Model
        self.systemMessage=systemMessage
        self.query=query
        self.Messages=[{'role': 'system', 'content': systemMessage},
                       {'role': 'user', 'content': query}]

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMResult:
    def __init__(self,UUID:str="",result:str="",errorMsg:str="",timeDelta:str=""):
        self.UUID=UUID
        self.result=result
        self.errorMsg=errorMsg
        self.HasError=errorMsg!=""
        self.timeDelta=timeDelta

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMRequestResultPair():
    def __init__(self,request:LLMRequest,result:LLMResult=None):
        self.uuid=request.UUID
        self.request=request
        self.result=result


In [11]:
#Main OLLAMA Calling Function / Example code which posts data to queue
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage



#Post to a service bus queue    
def AzurePost_ServiceBus(connection_string: str, queue_name: str, json_payload):
    with ServiceBusClient.from_connection_string(connection_string) as client:
        sender = client.get_queue_sender(queue_name=queue_name)
        message = ServiceBusMessage(json.dumps(json_payload))
        with sender:
            sender.send_messages(message)
            print(f"Queued message: {json_payload['UUID']}")

#Take a list of queries and turn them into results
def BatchTransForm(Model:str,systemMessage:str,queries:List[str],DynamicUI:bool=True,WebUI:bool=False,ShowWebUI:bool=False)->List[str]:
    
    #Put every prompt into the LLM_Requests pool
    LLMrequests=[]
    for query in queries:
        request=LLMRequest(Model,systemMessage,query)
        LLMrequests.append(request)
        AzurePost_ServiceBus(EndPoint_Queries,"llm-queries",request.to_json())
    
    #Monitor the results queue
    return LLMrequests
    
    #Output the results
    
#Test code
systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""
queuedRequests=BatchTransForm("mistral-small",systemPrompt,requestTexts)



Queued message: 6bdea878-7bcc-4f3d-b979-f0ba610d7091
Queued message: 71d41245-ee9b-44e9-b082-c01eff7f3a6c
Queued message: f013cea8-0ec3-4af6-9a5f-73fc1078d8e2
Queued message: 249c0b7f-8f1c-429c-87c5-fb340af9dd74
Queued message: 9d70d4c3-84ed-4374-8818-6d690b0f9b2a
Queued message: 8e8de587-e725-42d2-851a-faf7722d19af
Queued message: 8596841b-df89-4822-bb6a-73d2e8918bcc
Queued message: fc124937-8d17-43e5-9671-d90c278bcde6
Queued message: a082448a-2789-402e-bad6-98673ef8f8fd
Queued message: 1a4835f6-e8fc-4300-b115-78d5dee1e18d


In [12]:
#Main Result Handling Loop
import time
from typing import List
from azure.servicebus import ServiceBusClient, ServiceBusMessage

#Turn queud requests into dictionary
queueDict={}
for request in queuedRequests:
    queueDict[request.UUID]=LLMRequestResultPair(request)
totalResults=0
invalidResults=0

def ProcessResult(result:LLMResult):
    global totalResults
    global invalidResults
    if result.UUID in queueDict:
        print(f"Result UUID: {result.UUID} found in queueDict")
        queueDict[result.UUID].result=result
        totalResults+=1
    else:
        print(f"Result UUID: {result.UUID} not found in queueDict")
        invalidResults+=1

# Replace with your connection string and queue name
queue_name = "llm-responses"
servicebus_client = ServiceBusClient.from_connection_string(conn_str=Endpoint_Results)

def receive_messages_from_queue():
    # Create a receiver for the queue
    with servicebus_client.get_queue_receiver(queue_name=queue_name) as receiver:
        print("Receiving messages from the queue...")
        
        # Receive messages in a batch, you can specify max_message_count as needed
        received_msgs = receiver.receive_messages(max_message_count=10, max_wait_time=5)

        for msg in received_msgs:
            # Print the message payload
            print(f"Received message: {str(msg)}")
            receiver.complete_message(msg)
            
            llmRequest=LLMResult()
            llmRequest.from_json(str(msg))
            #llmRequest.from_json(json.loads(str(msg)))
            ProcessResult(llmRequest)
            
            # Accept the message to remove it from the queue
            #receiver.complete_message(msg)

# Call the function to read from the queue
while totalResults<len(queueDict):
    receive_messages_from_queue()
    print(f"Total Results: {totalResults}, Invalid Results: {invalidResults}")



Receiving messages from the queue...
Total Results: 0, Invalid Results: 0
Receiving messages from the queue...
Total Results: 0, Invalid Results: 0
Receiving messages from the queue...
Received message: {"UUID": "6bdea878-7bcc-4f3d-b979-f0ba610d7091", "result": {"model": "mistral-small", "created_at": "2024-09-23T20:44:12.012773698Z", "message": {"role": "assistant", "content": " ### Character Description and Background Story\n\n**Name:** Thoren Stonemantle\n\n**Race:** Gnome\n\n**Complexion:** Ruddy\n\nIn the heart of the ancient realm of Ealdholt, where the air was thick with the scent of earth and the whispers of ancient magic, there lived a gnome named Thoren Stonemantle. His complexion was as ruddy as the sunset over the Ironpeak Mountains, a testament to his days spent laboring in the forges and mines beneath the earth. Despite his diminutive stature, Thoren possessed an aura of rugged endurance that was both admired and envied by his kin.\n\n**Height:** Tall (for a gnome)\n\nFor

In [ ]:
#Edge Case -> delibrately fail a few requests

systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""

#Invalid model!
queuedRequests=BatchTransForm("herdepr28.1",systemPrompt,requestTexts[0:1])

